In [1]:
import sys
sys.path.append('/home/cdsw/utils')

In [2]:
from docx import Document
from llama_cpp import Llama

from copy import deepcopy
from llm_proofer import *
from prompts import (
    karen_prompt,
    karen_system_prompt,
    llama_prompt,
    llama_system_prompt,
    gemma_prompt,
)
from llm_configs import karen_config, llama_config, gemma_config

#set up the LLM
llm = Llama.from_pretrained(
    repo_id="bartowski/Qwen2.5-7B-Instruct-GGUF",
    filename="Qwen2.5-7B-Instruct-Q6_K_L.gguf",
    n_gpu_layers=-1,
    n_ctx=16384,
)

if "karen" in llm.model_path:
    generation_params = karen_config
    PROMPT_TEMPLATE = karen_prompt
    SYS_PROMPT = karen_system_prompt
elif "gemma" in llm.model_path:
    generation_params = gemma_config
    PROMPT_TEMPLATE = gemma_prompt
    SYS_PROMPT = None
else:
    generation_params = llama_config
    PROMPT_TEMPLATE = llama_prompt
    SYS_PROMPT = llama_system_prompt



/home/cdsw/.local/lib/python3.11/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
/home/cdsw/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 38 key-value pairs and 339 tensors from /home/cdsw/.cache/huggingface/hub/models--bartowski--Qwen2.5-7B-Instruct-GGUF/snapshots/8911e8a47f92bac19d6f5c64a2e2095bd2f7d031/./Qwen2.5-7B-Instruct-Q6_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              

In [3]:
from smartdoc_utils import create_track_changes_document,build_file_name_extensions
input_path = "/home/cdsw/data/simple-word-file-with-table.docx"

_,_, edit_path = build_file_name_extensions(input_path, extension="output")
_,_, correction_path = build_file_name_extensions(input_path, extension="corrections")
input_doc = Document(input_path)
modified_doc = deepcopy(input_doc)
# Create a document object for corrections
corrections_doc = Document()
modified_doc, corrections_doc = proofread_and_correct_document(
    modified_doc, corrections_doc
)
modified_doc.save(edit_path)
corrections_doc.save(correction_path)
new_doc = create_track_changes_document(input_path, edit_path)
print('new document located in:', new_doc)



comment_id:1
Added Comment to Document :  Spelling and grammar issues corrected.
comment_id:2
Added Comment to Document :  Font family was Times New Roman (expected: ['Calibri', 'Calibri Light'])
comment_id:3
Added Comment to Document :  Font family was Times New Roman (expected: ['Calibri', 'Calibri Light'])
comment_id:4
Added Comment to Document :  Font color was 00B0F0 (expected: one of ['000000', '0000FF'])
comment_id:5
Added Comment to Document :  Font family was Times New Roman (expected: ['Calibri', 'Calibri Light'])
comment_id:6
Added Comment to Document :  Font family was Times New Roman (expected: ['Calibri', 'Calibri Light'])
comment_id:7
Added Comment to Document :  Spelling and grammar issues corrected.
comment_id:8
Added Comment to Document :  Font size was 16.0 (expected: 12)
comment_id:9
Added Comment to Document :  Font family was Times New Roman (expected: ['Calibri', 'Calibri Light'])
comment_id:10
Added Comment to Document :  Font family was Times New Roman (expecte

llama_perf_context_print:        load time =   24909.17 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1705 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   26597.04 ms /  1730 tokens
Llama.generate: 1686 prefix-match hit, remaining 10 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

llama_perf_context_print:        load time =   24909.17 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    10 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    16 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1258.24 ms /    26 tokens
Llama.generate: 1686 prefix-match hit, remaining 8 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

...

PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

llama_perf_context_print:        load time =   24909.17 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    10 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    16 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1265.46 ms /    26 tokens
Llama.generate: 1686 prefix-match hit, remaining 10 prompt tokens to eval


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 

llama_perf_context_print:        load time =   24909.17 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    10 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    16 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1251.64 ms /    26 tokens


PROMPT: 
 Use the following examples as reference for editing.

------------------------------------------------------------
Example 1.
------------------------------------------------------------

Text: 
'I saw a    sample prodcut.  These product are below expeccted quality. I start to underdstand what he said is quite right. The correct valu is 12.5%% and 3rd postion. The time is 10:30am. The length of the object is 845mm and costs $1million. The sunise happens at 0750 EST and visibilty is 35 km. The right way too sumize and favor some advisor's is still to be found.'

1. Edited Text: 'I saw a sample product. These products are below expected quality. I started to understand what he said is quite right. The correct value is 12.5%% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors' is still to be found.'
2. Corrections:
   a) Removed extra 